### Part 1 - start

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
!pip install bs4 
#!pip install request for BeautifulSoup

In [3]:
from bs4 import BeautifulSoup # this module helps in web scrapping
import requests # this module helps to download a web page

In [4]:
#url contains an html table with Toronto postal codes data
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [5]:
# get the contents of the webpage in text format and store in a variable called data
data = requests.get(url).text
# create a Beautiful soup object
soup = BeautifulSoup(data,"html5lib")

In [6]:
#create a list  
table_contents=[]
#find the table and table data
table=soup.find('table')

In [7]:
#create a dictionary called cell having 3 keys: PostalCode, Borough, and Neighborhood
#as postal code contains up to 3 characters, extract that using tablerow.p.text
#use split, strip, and replace functions for getting Borough and Neighborhood information
#append to the list
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#create a dataframe with list table_contents
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

#print dataframe head
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [8]:
df.shape

(103, 3)

### Part 1 - end
### Part 2 - start

In [9]:
!pip install geocoder

In [10]:
import geocoder #import geocoder

#use a while loop to resolve latitude and longitude for postal codes
postal_code = df['PostalCode']
latitude = []
longitude = []
n = 0

while n < len(postal_code):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code[n]))
    lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    n = n + 1

#add latitude and longitude data to dataframe
df['Latitude'] = latitude
df['Longitude'] = longitude

#print dataframe head
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


In [11]:
df.shape

(103, 5)

### Part 2 - end
### Part 3 - start

#### map and explore Toronto neighborhoods from dataframe with Foursquare API and folium

In [16]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'LQKDVVN24XV3FA0KSFELBKJHJ2M0MPKPQZ4INSEQHD5E3XHL' # your Foursquare ID
CLIENT_SECRET = '1QZAABBJTO1MPZ2U3BZIGY2B40QKPIUBO2BJFOZVCTIMAW0Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LQKDVVN24XV3FA0KSFELBKJHJ2M0MPKPQZ4INSEQHD5E3XHL
CLIENT_SECRET:1QZAABBJTO1MPZ2U3BZIGY2B40QKPIUBO2BJFOZVCTIMAW0Q


In [17]:
# get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto's","latitude is",latitude_downtown_toronto, "& longitude is" ,longitude_downtown_toronto)

Downtown Toronto's latitude is 43.6563221 & longitude is -79.3809161


In [18]:
# create map of Toronto neighborhoods
map_toronto = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### explore Toronto neighborhoods using Foursquare API

In [19]:
#create a function to get nearby venues in Toronto neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
#run the above function on each Toronto neighborhood, storing result in toronto_venues dataframe
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [21]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.73057,-79.31306,Stellar's Lenonade Stand,43.731507,-79.311712,Food Stand
3,Victoria Village,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store


In [22]:
#check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,13,13,13,13,13,13
"Alderwood, Long Branch",4,4,4,4,4,4
"Bathurst Manor, Wilson Heights, Downsview North",2,2,2,2,2,2
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Berczy Park,64,64,64,64,64,64
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",84,84,84,84,84,84
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",78,78,78,78,78,78


In [23]:
#find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 267 uniques categories.


#### analyze each neighborhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# shift column 'Neighborhood' to first position
first_column = toronto_onehot.pop('Neighborhood')
# insert column using insert(position,column_name, first_column) function
toronto_onehot.insert(0, 'Neighborhood', first_column)

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [25]:
#group rows by neighborhood and display the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.076923,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.

In [26]:
#print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                   venue  freq
0   Hong Kong Restaurant  0.08
1           Skating Rink  0.08
2         Discount Store  0.08
3       Department Store  0.08
4  Vietnamese Restaurant  0.08


----Alderwood, Long Branch----
                venue  freq
0  Athletics & Sports  0.25
1   Convenience Store  0.25
2                 Pub  0.25
3                 Gym  0.25
4              Museum  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0        Men's Store   0.5
1   Business Service   0.5
2  Accessories Store   0.0
3             Museum   0.0
4          Nightclub   0.0


----Bayview Village----
                        venue  freq
0                       Trail  0.50
1  Construction & Landscaping  0.25
2                        Park  0.25
3           Accessories Store  0.00
4                      Museum  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.10
1      Sandwich Place  0.10
2  It

In [27]:
#put the above into a pandas dataframe
#first, define a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Discount Store,Badminton Court,Bubble Tea Shop,Sushi Restaurant,Hong Kong Restaurant,Vietnamese Restaurant,Skating Rink,Shopping Mall,Department Store,Supermarket
1,"Alderwood, Long Branch",Convenience Store,Athletics & Sports,Pub,Gym,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Men's Store,Business Service,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
3,Bayview Village,Trail,Park,Construction & Landscaping,Falafel Restaurant,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Zoo Exhibit
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pub,Thai Restaurant,Indian Restaurant,Sports Club,Liquor Store,Sushi Restaurant


#### cluster neighborhoods

In [28]:
#run k-means to cluster the neighborhood into 5 clusters
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 4, 0, 3, 3, 3, 3, 3, 3])

In [29]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge manhattan_marged with df to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.75245,-79.32991,1.0,Food & Drink Shop,Park,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Eastern European Restaurant
1,M4A,North York,Victoria Village,43.73057,-79.31306,0.0,Grocery Store,Park,Nail Salon,Food Stand,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Zoo Exhibit
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,3.0,Coffee Shop,Breakfast Spot,Health Food Store,Theater,Playground,Distribution Center,Pub,Restaurant,Electronics Store,Event Space
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,3.0,Clothing Store,Furniture / Home Store,Food Court,Toy / Game Store,Women's Store,American Restaurant,Men's Store,Cosmetics Shop,Bookstore,Fried Chicken Joint
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188,3.0,Coffee Shop,Bank,Burrito Place,Café,Falafel Restaurant,Sandwich Place,Fried Chicken Joint,Gastropub,Italian Restaurant,Theater
5,M9A,Etobicoke,Islington Avenue,43.66263,-79.52831,3.0,Pharmacy,Café,Park,Shopping Mall,Skating Rink,Grocery Store,Bank,Farm,Farmers Market,Electronics Store
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662,3.0,Zoo Exhibit,Construction & Landscaping,Fast Food Restaurant,Falafel Restaurant,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Farmers Market
7,M3B,North York,Don Mills North,43.74923,-79.36186,0.0,Burger Joint,Gas Station,Soccer Field,Park,Coffee Shop,Falafel Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192,3.0,Pizza Place,Athletics & Sports,Fast Food Restaurant,Café,Breakfast Spot,Rock Climbing Spot,Gastropub,Bank,Intersection,Flea Market
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,3.0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Hotel,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Theater,Movie Theater


In [30]:
#drop rows from toronto_merged that have null/NaN value
print ("Shape of dataset before cleaning: ", toronto_merged.size)
toronto_merged.dropna(axis=0, inplace = True)
print ("Shape of dataset after cleaning: ", toronto_merged.size)

Shape of dataset before cleaning:  1648
Shape of dataset after cleaning:  1616


In [31]:
# reset index for toronto_merged dataframe since droped rows
toronto_merged.reset_index(drop=True, inplace=True)
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.75245,-79.32991,1.0,Food & Drink Shop,Park,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Eastern European Restaurant
1,M4A,North York,Victoria Village,43.73057,-79.31306,0.0,Grocery Store,Park,Nail Salon,Food Stand,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Zoo Exhibit
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,3.0,Coffee Shop,Breakfast Spot,Health Food Store,Theater,Playground,Distribution Center,Pub,Restaurant,Electronics Store,Event Space
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,3.0,Clothing Store,Furniture / Home Store,Food Court,Toy / Game Store,Women's Store,American Restaurant,Men's Store,Cosmetics Shop,Bookstore,Fried Chicken Joint
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188,3.0,Coffee Shop,Bank,Burrito Place,Café,Falafel Restaurant,Sandwich Place,Fried Chicken Joint,Gastropub,Italian Restaurant,Theater
5,M9A,Etobicoke,Islington Avenue,43.66263,-79.52831,3.0,Pharmacy,Café,Park,Shopping Mall,Skating Rink,Grocery Store,Bank,Farm,Farmers Market,Electronics Store
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662,3.0,Zoo Exhibit,Construction & Landscaping,Fast Food Restaurant,Falafel Restaurant,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Farmers Market
7,M3B,North York,Don Mills North,43.74923,-79.36186,0.0,Burger Joint,Gas Station,Soccer Field,Park,Coffee Shop,Falafel Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192,3.0,Pizza Place,Athletics & Sports,Fast Food Restaurant,Café,Breakfast Spot,Rock Climbing Spot,Gastropub,Bank,Intersection,Flea Market
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,3.0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Hotel,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Theater,Movie Theater


In [32]:
#visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### examine clusters

#### 1st cluster - grocery, restaurants, recreation (e.g., parks, trails, playgrounds)

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Grocery Store,Park,Nail Salon,Food Stand,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Zoo Exhibit
7,North York,0.0,Burger Joint,Gas Station,Soccer Field,Park,Coffee Shop,Falafel Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop
16,York,0.0,Playground,Trail,Grocery Store,Field,Park,Hockey Arena,Art Gallery,Cuban Restaurant,Flower Shop,Flea Market
18,Scarborough,0.0,Construction & Landscaping,Gym / Fitness Center,Park,Zoo Exhibit,Falafel Restaurant,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop
22,Scarborough,0.0,Korean BBQ Restaurant,Business Service,Coffee Shop,Park,Falafel Restaurant,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop
31,Scarborough,0.0,Spa,Indian Restaurant,Restaurant,Park,Grocery Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop
34,East York/East Toronto,0.0,Park,Fabric Shop,Intersection,Eastern European Restaurant,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Zoo Exhibit
38,North York,0.0,Trail,Park,Construction & Landscaping,Falafel Restaurant,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Zoo Exhibit
48,North York,0.0,Bakery,Basketball Court,Park,Trail,Falafel Restaurant,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop
56,North York,0.0,Coffee Shop,Park,Zoo Exhibit,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Eastern European Restaurant


#### 2nd cluster - parks & zoo

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Food & Drink Shop,Park,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Eastern European Restaurant
26,North York,1.0,Park,Residential Building (Apartment / Condo),Falafel Restaurant,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Zoo Exhibit,Eastern European Restaurant
44,North York,1.0,Park,Zoo Exhibit,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Eastern European Restaurant
67,Central Toronto,1.0,Park,Zoo Exhibit,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Eastern European Restaurant


#### 3rd cluster - home service & stores

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,North York,2.0,Home Service,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
61,Central Toronto,2.0,Home Service,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm


#### 4th cluster - food, shopping, and recreation venues

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3.0,Coffee Shop,Breakfast Spot,Health Food Store,Theater,Playground,Distribution Center,Pub,Restaurant,Electronics Store,Event Space
3,North York,3.0,Clothing Store,Furniture / Home Store,Food Court,Toy / Game Store,Women's Store,American Restaurant,Men's Store,Cosmetics Shop,Bookstore,Fried Chicken Joint
4,Queen's Park,3.0,Coffee Shop,Bank,Burrito Place,Café,Falafel Restaurant,Sandwich Place,Fried Chicken Joint,Gastropub,Italian Restaurant,Theater
5,Etobicoke,3.0,Pharmacy,Café,Park,Shopping Mall,Skating Rink,Grocery Store,Bank,Farm,Farmers Market,Electronics Store
6,Scarborough,3.0,Zoo Exhibit,Construction & Landscaping,Fast Food Restaurant,Falafel Restaurant,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Farmers Market
8,East York,3.0,Pizza Place,Athletics & Sports,Fast Food Restaurant,Café,Breakfast Spot,Rock Climbing Spot,Gastropub,Bank,Intersection,Flea Market
9,Downtown Toronto,3.0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Hotel,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Theater,Movie Theater
10,North York,3.0,Pizza Place,Grocery Store,Italian Restaurant,Fast Food Restaurant,Latin American Restaurant,Sushi Restaurant,Gas Station,Bank,Bakery,Japanese Restaurant
11,Etobicoke,3.0,Pizza Place,Tea Room,Sandwich Place,Chinese Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Fish & Chips Shop,Fabric Shop
12,Scarborough,3.0,Home Service,Construction & Landscaping,Bar,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Elementary School,Farmers Market


#### 5th cluster - shopping, markets

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,North York,4.0,Men's Store,Business Service,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
